In [3]:
import tensorflow as tf
import numpy as np

In [4]:
import pickle
with open("tokenizer_data.pkl", 'rb') as f:
    data = pickle.load(f)
    tokenizer = data['tokenizer']
    num_words = data['num_words']
    max_sequence_len = data['max_len']

In [5]:
from tensorflow.keras.models import load_model
model = load_model('tg.h5')

2023-07-27 15:37:46.529376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 15:37:46.643038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 15:37:46.643084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 15:37:46.644625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-27 15:37:46.644667: I tensorflow/compile

In [6]:
with open('data.txt', 'r', encoding='utf-8') as file:
    data= file.read()

In [7]:
vocab = sorted(set(data))
print(f'{len(vocab)} unique characters')

68 unique characters


In [8]:
data=data.split('\n')

In [9]:
print(tokenizer.word_index)

{' ': 1, 'e': 2, 'a': 3, 'i': 4, 'n': 5, 'r': 6, 't': 7, 's': 8, 'o': 9, 'c': 10, 'm': 11, 'd': 12, 'l': 13, 'h': 14, 'y': 15, '\n': 16, 'p': 17, 'g': 18, 'f': 19, 'u': 20, '.': 21, 'v': 22, "'": 23, '(': 24, ')': 25, 'w': 26, 'b': 27, ',': 28, 'k': 29, 'j': 30, '-': 31, 'q': 32, 'z': 33, 'x': 34, '1': 35, '9': 36, '"': 37, '&': 38, '3': 39, '5': 40}


In [10]:
input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    token_list = token_list[:max_sequence_len]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [11]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

148

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences =np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [13]:
xs= input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [14]:
xs

array([[ 0,  0,  0, ...,  0,  0,  3],
       [ 0,  0,  0, ...,  0,  3,  1],
       [ 0,  0,  0, ...,  3,  1, 10],
       ...,
       [ 0,  0, 11, ...,  6, 18,  2],
       [ 0, 11, 20, ..., 18,  2,  6],
       [11, 20, 10, ...,  2,  6,  1]], dtype=int32)

In [15]:
ys = tf.keras.utils.to_categorical(labels, num_classes=41)

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
history = model.fit(xs,ys,epochs=5)

Epoch 1/5
175/175 [==============================] - 29s 165ms/step - loss: 0.3222 - accuracy: 0.9480
Epoch 2/5
175/175 [==============================] - 31s 175ms/step - loss: 0.2807 - accuracy: 0.9575
Epoch 3/5
175/175 [==============================] - 30s 170ms/step - loss: 0.2452 - accuracy: 0.9679
Epoch 4/5
175/175 [==============================] - 29s 167ms/step - loss: 0.2121 - accuracy: 0.9760
Epoch 5/5
175/175 [==============================] - 29s 163ms/step - loss: 0.1838 - accuracy: 0.9817


In [21]:
model.save('tg-v2.h5')